![rmotr](https://i.imgur.com/jiPp4hj.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/39117440-24199c72-46e7-11e8-8ffc-25c6e27e07d4.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Handling Missing Data with Pandas

pandas borrows all the capabilities from numpy selection + adds a number of convenient methods to handle missing values. Let's see one at a time:

![separator2](https://i.imgur.com/4gX5WFr.png)

## Hands on! 

In [120]:
import numpy as np
import pandas as pd

In [121]:
# For numpy

np.isnan(np.nan), np.isinf(np.inf), np.isfinite(np.nan), np.isfinite(np.inf)

(True, True, False, False)

### Pandas utility functions

Similarly to `numpy`, pandas also has a few utility functions to identify and detect null values:

In [122]:
""" 
    Detect missing values for an array-like object.

    This function takes a scalar or array-like object and indictates
    whether values are missing (``NaN`` in numeric arrays, ``None`` or ``NaN``
    in object arrays, ``NaT`` in datetimelike).
   
"""   
pd.isna(np.nan)

True

In [123]:
pd.isna(None)

True

In [124]:
pd.isna('app')

False

**isnull is an alias for isna.**

\>>> pd.isnull

<function isna at 0x7fb4c5cefc80>

---
So isna is the better choice!

In [125]:
pd.isnull(np.nan)

True

In [126]:
pd.isnull(None)

True

The opposite ones also exist:

In [127]:
pd.notna(np.nan)

False

In [128]:
pd.notnull(None)

False

In [129]:
pd.notnull(np.nan)

False

In [130]:
pd.notnull(3)

True

These functions also work with Series and `DataFrame`s, and return the same types:

In [131]:
pd.isna(pd.Series([1, np.nan, 7]))

0    False
1     True
2    False
dtype: bool

In [132]:
pd.notna(pd.Series([1, np.nan, 7]))

0     True
1    False
2     True
dtype: bool

In [133]:
pd.isna(pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
}))

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


In [134]:
test_df = pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
})

test_df.isna()

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


In [135]:
pd.isna(test_df)

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


![separator1](https://i.imgur.com/ZUWYTii.png)

### Pandas Operations with Missing Values

Pandas manages missing values more gracefully than numpy. `nan`s will no longer behave as "viruses", and **operations will just ignore them completely:**

REMEMBER: nan is STILL from numpy, so it's np.nan, eventhough used with a pd method here.

In [136]:
pd.Series([1, 2, np.nan]).count()

2

In [137]:
pd.Series([1, 2, np.nan]).sum()

3.0

In [138]:
pd.Series([2, 2, np.nan]).mean()

2.0

### Filtering missing data

As we saw with numpy, we could combine boolean selection + `pd.isna` to filter out those `nan`s and null values:

In [139]:
s = pd.Series([1, 2, 3, np.nan, np.nan, 4])

In [140]:
pd.notna(s)

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [141]:
pd.isna(s)

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [142]:
# Essentially counting non-null values

pd.notna(s).sum()

4

In [143]:
# Essentially counting null values

pd.isna(s).sum()

2

In [144]:
s[pd.notna(s)]

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

But both `notnull` and `isnull` are also methods of `Series` and `DataFrame`s, so we could use it that way:

In [145]:
s.isna()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

In [146]:
s.notna()

0     True
1     True
2     True
3    False
4    False
5     True
dtype: bool

In [147]:
s[s.notna()].sum()

10.0

In [148]:
# But since s is a pd.Series, it effectively ignores the np.nan values, therefore the above filtering is not strictly required

s.sum()

10.0

In [149]:
# But for p, which is a np.array, this is not the case. nan will act like a virus and not be ignored in the opetation.

p = np.array([1, 2, 3, np.nan, np.nan, 4])

p.sum()

nan

![separator1](https://i.imgur.com/ZUWYTii.png)

### Dropping null values

Boolean selection + `notna()` seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the `dropna` method:

In [150]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

In [151]:
s.dropna()

0    1.0
1    2.0
2    3.0
5    4.0
dtype: float64

### Dropping null values on DataFrames

You saw how simple it is to drop `na`s with a Series. 

But with `DataFrame`s, there will be a few more things to consider, because **you can't drop single values.**

**You can only drop entire columns or rows.**

Let's start with a sample `DataFrame`:

In [152]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
}, index = ['a','b','c','d'])

In [153]:
df

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,NaN,8.0,9.0,8
c,30.0,31.0,32.0,34
d,NaN,NaN,100.0,110


In [154]:
df.shape

(4, 4)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, a to d
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 160.0+ bytes


In [156]:
df.isna()

,Column A,Column B,Column C,Column D
a,False,False,True,False
b,True,False,False,False
c,False,False,False,False
d,True,True,False,False


In [157]:
# Counting null values

# df.isna().count() gives an incorrect result since BOTH true and false values are counted

# Whereas sum() takes into account only the true values, akin to 1

df.isna().sum()

Column A    2
Column B    1
Column C    1
Column D    0
dtype: int64

The default `dropna` behavior will **drop all the rows in which _any_ null value is present:**

In [158]:
df.dropna()

,Column A,Column B,Column C,Column D
c,30.0,31.0,32.0,34


In this case we're dropping **rows**. Rows containing null values are dropped from the DF. You can also use the `axis` parameter to drop columns containing null values:

In [159]:
df.dropna(axis = 1)  # axis = 'columns' also works

,Column D
a,5
b,8
c,34
d,110


In this case, any row or column that contains **at least** one null value will be dropped. Which can be, depending on the case, too extreme. 

You can control this behavior with the `how` parameter. Can be either `'any'` or `'all'`:

In [160]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [161]:
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [162]:
# Drop a row (or column) only if ALL values in it are NaN. Others are retained

df2.dropna(how = 'all')

,Column A,Column B,Column C
0,1.0,2.0,NaN
2,30.0,31.0,100.0


In [163]:
# 'any' drops a row (or column) if even one value in it is NaN, as is the default behaviour

df2.dropna(how = 'any')

,Column A,Column B,Column C
2,30.0,31.0,100.0


You can also use the `thresh` parameter to indicate a _threshold_ (a minimum number) of **non-null values** for the row/column to be kept:

In [164]:
df

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,NaN,8.0,9.0,8
c,30.0,31.0,32.0,34
d,NaN,NaN,100.0,110


In [165]:
df.dropna(thresh = 3)

# Only 2 non-null values in row d, there fore it is dropped

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,NaN,8.0,9.0,8
c,30.0,31.0,32.0,34


In [193]:
df.dropna(thresh = 3, axis = 'columns')

,Column B,Column C,Column D
a,2.0,NaN,5
b,8.0,9.0,8
c,31.0,32.0,34
d,NaN,100.0,110


![separator1](https://i.imgur.com/ZUWYTii.png)

### Filling null values (on Series)

Sometimes instead of dropping the null values, we might need to **replace them** with some other value. 

NOTE: This highly depends on your context and the dataset you're currently working on. 

A `nan` can be replaced with
* `0`
* the `mean` of the sample
* the closest value

Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem in the most suitable manner.

In [167]:
s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

**Filling nulls with a arbitrary value**

In [198]:
# Filling with 0

s.fillna(0)

0    1.0
1    2.0
2    3.0
3    0.0
4    0.0
5    4.0
dtype: float64

In [201]:
# Filling with the mean of the non-null value set (1 + 2 + 3 + 4)/4 = 10/4 = 2.5

s.fillna(s.mean())

0    1.0
1    2.0
2    3.0
3    2.5
4    2.5
5    4.0
dtype: float64

In [194]:
# Original set remains the same

s

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64

**Filling nulls with contiguous (close) values**

The `method` argument is used to fill null values with other values close to that null one.

method='ffill': Ffill or forward-fill propagates the last observed non-null value forward until another non-null value is encountered. 

method='bfill': Bfill or backward-fill propagates the first observed non-null value backward until another non-null value is met.

In [171]:
s.fillna(method = 'ffill')

0    1.0
1    2.0
2    3.0
3    3.0
4    3.0
5    4.0
dtype: float64

In [172]:
s.fillna(method = 'bfill')

0    1.0
1    2.0
2    3.0
3    4.0
4    4.0
5    4.0
dtype: float64

**NOTE:** This can still leave null values at the extremes of the Series/DataFrame:

In [200]:
# No value before the NaN at the very beginning 

pd.Series([np.nan, 3, np.nan, 9]).fillna(method = 'ffill')

0    NaN
1    3.0
2    3.0
3    9.0
dtype: float64

In [199]:
# No value after the NaN at the very end

pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method = 'bfill')

0    1.0
1    3.0
2    3.0
3    NaN
4    NaN
dtype: float64

### Filling null values on DataFrames

The `fillna` method also works on `DataFrame`s, and it works similarly. 

The main differences are that you can **specify the `axis`** (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [175]:
df

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,NaN,8.0,9.0,8
c,30.0,31.0,32.0,34
d,NaN,NaN,100.0,110


In [202]:
# Specifying different fill values for each column

df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
a,1.0,2.0,47.0,5
b,0.0,8.0,9.0,8
c,30.0,31.0,32.0,34
d,0.0,99.0,100.0,110


In [177]:
# Going down a column, the first value of C remains unfilled since no value before it

df.fillna(method = 'ffill', axis=0)

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,1.0,8.0,9.0,8
c,30.0,31.0,32.0,34
d,30.0,31.0,100.0,110


In [207]:
# Going across a row, the first values of b & c remains unfilled since no value before it

df.fillna(method='ffill', axis=1)

,Column A,Column B,Column C,Column D
a,1.0,2.0,2.0,5.0
b,NaN,8.0,9.0,8.0
c,30.0,31.0,32.0,34.0
d,NaN,NaN,100.0,110.0


In [206]:
# REctified through use of method = 'bfill'

df.fillna(method='bfill', axis=1)

,Column A,Column B,Column C,Column D
a,1.0,2.0,5.0,5.0
b,8.0,8.0,9.0,8.0
c,30.0,31.0,32.0,34.0
d,100.0,100.0,100.0,110.0


![separator1](https://i.imgur.com/ZUWYTii.png)

### Checking if there are NAs

The question is: Does this `Series` or `DataFrame` contain any missing value? The answer should be yes or no: `True` or `False`. How can you verify it?

**For a Series**

If there are missing values, `s.dropna()` will have less elements than `s`:

In [228]:
# s is Series and count function automatically removes null values
# But len counts the null values too

print(s)
print('count: ', s.count())
print('len: ',len(s))

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64
count:  4
len:  6


In [227]:
# Cheking for a Series

missing_values_series = len(s.dropna()) != len(s)
missing_values

True

Since `count` method disregards nulls, we can just use the below wxpression for a Series:

In [229]:
missing_values_series = s.count() != len(s)
missing_values

True

**For a DataFrame:**

In [224]:
# len function counts the number of rows (including null values) in the dataframe

len(df)

4

In [225]:
# count methods returns non-null values in each column of the DataFrame
# Therefore len() and .count() values cannot be compared

df.count()

Column A    2
Column B    3
Column C    3
Column D    4
dtype: int64

In [231]:
# Compare len values between original and dropped df's

missing_values_dataframe = len(df.dropna()) != len(df)

missing_values_dataframe

True

#### More Pythonic solution `any`

**For a Series:**

The method `any` checks if there's at least one `True` values in a Series. 

Similarly, the method `all` checks if _all_ the values are `True`. 

They work in the same way as in Python:

In [184]:
pd.Series([True, False, False]).any()

True

In [185]:
pd.Series([True, False, False]).all()

False

In [186]:
pd.Series([True, True, True]).all()

True

The `isna()` method returned a Boolean `Series` with `True` values wherever there was a `nan`:

In [232]:
s.isna()

0    False
1    False
2    False
3     True
4     True
5    False
dtype: bool

So we can just use the `any` method with the boolean array returned:

In [235]:
test_df1 = pd.Series([1, np.nan])

test_df1.isna().any()

True

In [236]:
test_df2 = pd.Series([1, 2])

test_df2.isna().any()

False

In [238]:
print(s)

s.isna().any()

0    1.0
1    2.0
2    3.0
3    NaN
4    NaN
5    4.0
dtype: float64


True

A more strict version would check only the `values` of the Series:

In [239]:
s.isna().values

array([False, False, False,  True,  True, False])

In [192]:
s.isna().values.any()

True

**For a DataFrame:**

In [240]:
df

,Column A,Column B,Column C,Column D
a,1.0,2.0,NaN,5
b,NaN,8.0,9.0,8
c,30.0,31.0,32.0,34
d,NaN,NaN,100.0,110


In [243]:
# For null values present in a DataFrame, it makes sense to know exactly in which column they are present
# Therefore the result returned by the any() method is insightful

df.isna().any()

Column A     True
Column B     True
Column C     True
Column D    False
dtype: bool

In [246]:
# can we check across rows?



![separator2](https://i.imgur.com/4gX5WFr.png)